<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/TestingStatsNFeaturesCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Refactor GatheringBaseMetrics

Used to test code built up/ worked out in the following notebook...
* https://github.com/curtiscu/LYIT/blob/master/GatheringBaseMetrics_1.ipynb

# Setup env


In [1]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 42
-rw------- 1 root root 21765 Jun 26 11:24 data_prep.py
drwx------ 2 root root  4096 Jun 26 14:53 __pycache__
-rw------- 1 root root  7272 Jun 26 14:53 stats_n_features.py
-rw------- 1 root root  8416 Jun  2 21:24 visualizations.py


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [4]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [5]:
# install required libs
!pip install mido

In [6]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep as dp

>> LOADING custom module, when: 2020-06-26 14:59:12.601089, module name: data_prep


In [7]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [8]:
# testing auto reload of modules 
dp.test_function_call('bling')

Test function in data_prep.py called and worked! when: 2020-06-26 14:59:12.647333,  param:bling


In [9]:
import stats_n_features as sf

>> LOADING custom module, when: 2020-06-26 14:59:12.672063, module name: stats_n_features


In [10]:
sf.test_function_call('hello')

Test function in stats_n_features called and worked! when: 2020-06-26 14:59:12.694519,  param:hello


## Pandas display options

In [11]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Testing stats_n_features.py

## Load some data

In [12]:
file_1 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'
file_7 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer7/eval_session/1_funk-groove1_138_beat_4-4.mid'

file_df, f, mtt = dp.load_file(file_1)  # load data

FILE name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
    loaded file: <midi file '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid' type 0, 1 tracks, 1300 messages>
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    track count: 1, tracks: [<midi track 'MIDI' 1300 messages>]
    MIDI file type: 0
    > processing track: <midi track 'MIDI' 1300 messages>
    note_on span - first tick: 3 , last tick: 30658 
    good instruments: 6, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 42.0: 'Closed Hi Hat (42)', 43.0: 'High Floor Tom (43)', 49.0: 'Crash Cymbal 1 (49)', 51.0: 'Ride Cymbal 1 (51)'}
    bar info - bars in file: 16, bar quantize level: 16.0
    tick info - ticks per time sig beat: 480.0, ticks per quantize bin: 120


## (1) Test sf.frequency_table()

In [13]:
freq_table = sf.frequency_table(file_df)

In [14]:
display(freq_table)

bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,,,,,,,,,,,,,,,,
Bass Drum 1 (36),12,0,16,4,0,0,0,0,0,0,15,6,0,1,0,0
Acoustic Snare (38),10,5,0,0,16,0,0,16,0,17,0,2,8,0,8,19
Closed Hi Hat (42),15,2,11,0,16,2,14,0,17,3,13,0,16,1,13,1
High Floor Tom (43),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Crash Cymbal 1 (49),1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
Ride Cymbal 1 (51),16,0,16,3,16,0,16,0,17,0,12,0,16,0,16,0


## (2) Test sf.error_buckets()

In [15]:
err_bucks = sf.error_buckets(file_df)

In [16]:
display(err_bucks)

track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
1          9               42            0:41          908        77   
                           51            0:43          940        46   
                           42            0:45          971         0   
                           51            0:46          971         0   
2          16              38           0:155         3710        37   
                           38           0:158         3747        41   
3          1               38           0:161         3796        33   
                           38           0:168         3846        23   
4          10              38           0:289         6813        37   
                           38           0:290         6867        20   
5          16              38           0:401         9468        37   
                           38           0:404         9506        33   
                           38           0:406         9532        30   
6          16              38           0:487        11397        36   
                           38           0:491        11441        32   
9          16              38           0:730        17147        38   
                           38           0:733        17182        38   
10         16              38           0:814        19056        43   
                           38           0:817        19092        46   
                           38           0:820        19116        40   
13         16              38          0:1056        24797        33   
                           38          0:1059        24845        32   
                           38          0:1062        24876        32   
14         16              38          0:1140        26733        40   
                           38          0:1143        26778        41   
                           38          0:1145        26805        34   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          9               42            -52         960           1   
                           51            -20         960           1   
                           42             11         960           1   
                           51             11         960           1   
2          16              38            -10        3720           2   
                           38             27        3720           2   
3          1               38            -44        3840           3   
                           38              6        3840           3   
4          10              38            -27        6840           4   
                           38             27        6840           4   
5          16              38            -12        9480           5   
                           38             26        9480           5   
                           38             52        9480           5   
6          16              38             -3       11400           6   
                           38             41       11400           6   
9          16              38            -13       17160           9   
                           38             22       17160           9   
10         16              38            -24       19080          10   
                           38             12       19080          10   
                           38             36       19080          10   
13         16              38            -43       24840          13   
                           38              5       24840          13   
                           38             36       24840          13   
14         16              38            -27       26760          14   
                           38             18       26760          14   
                           38             45       26760          14   

In [17]:
err_bucks.sample(3)

,,,track_msg_num,total_ticks,velocity,beat_offset,beat_center,bar_number,bar_beat_number
bar_number,bar_beat_number,note,,,,,,,
13,16,38,0:1062,24876,32,36,24840,13,16
5,16,38,0:401,9468,37,-12,9480,5,16
14,16,38,0:1143,26778,41,18,26760,14,16


## (3) Test sf.gather_stats()

In [18]:
stats_dict = sf.gather_stats(file_df)

In [19]:
display(stats_dict)

bar_beat_number     1      2       3      4       5      6      7       8   \
note feature                                                                 
36   hits        12.00   0.00   16.00   4.00    0.00   0.00   0.00    0.00   
     off_iqr     18.00   0.00   17.00  21.00    0.00   0.00   0.00    0.00   
     off_max     10.00   0.00    6.00   5.00    0.00   0.00   0.00    0.00   
     off_mean    -9.25   0.00  -10.56  -8.00    0.00   0.00   0.00    0.00   
     off_median  -8.00   0.00  -10.00  -8.50    0.00   0.00   0.00    0.00   
     off_min    -25.00   0.00  -29.00 -20.00    0.00   0.00   0.00    0.00   
     off_std     12.14   0.00   10.75  12.36    0.00   0.00   0.00    0.00   
     vel_iqr      8.00   0.00    9.00   5.00    0.00   0.00   0.00    0.00   
     vel_max     69.00   0.00  125.00  57.00    0.00   0.00   0.00    0.00   
     vel_mean    60.50   0.00   61.50  53.50    0.00   0.00   0.00    0.00   
     vel_median  61.50   0.00   60.50  54.50    0.00   0.00   0.00    0.00   
     vel_min     50.00   0.00   42.00  48.00    0.00   0.00   0.00    0.00   
     vel_std      5.45   0.00   18.30   3.87    0.00   0.00   0.00    0.00   
38   hits        10.00   5.00    0.00   0.00   16.00   0.00   0.00   16.00   
     off_iqr     39.00  22.00    0.00   0.00   10.50   0.00   0.00   18.50   
     off_max     55.00  -3.00    0.00   0.00   16.00   0.00   0.00   10.00   
     off_mean   -20.80 -21.40    0.00   0.00   -4.94   0.00   0.00  -11.69   
     off_median -41.00 -18.00    0.00   0.00   -6.00   0.00   0.00  -12.00   
     off_min    -58.00 -38.00    0.00   0.00  -24.00   0.00   0.00  -44.00   
     off_std     42.20  14.84    0.00   0.00   10.04   0.00   0.00   14.99   
     vel_iqr      8.00   6.00    0.00   0.00    9.00   0.00   0.00   17.50   
     vel_max     34.00  39.00    0.00   0.00  111.00   0.00   0.00  117.00   
     vel_mean    28.80  35.40    0.00   0.00  100.06   0.00   0.00   88.44   
     vel_median  29.50  38.00    0.00   0.00  101.00   0.00   0.00   97.50   
     vel_min     23.00  30.00    0.00   0.00   88.00   0.00   0.00   32.00   
     vel_std      4.49   4.10    0.00   0.00    6.48   0.00   0.00   23.66   
42   hits        15.00   2.00   11.00   0.00   16.00   2.00  14.00    0.00   
     off_iqr     21.00   0.00   12.50   0.00   12.00   0.00   9.00    0.00   
     off_max    -10.00  58.00  -21.00   0.00   -2.00  55.00 -16.00    0.00   
     off_mean   -28.93  51.50  -37.36   0.00  -24.38  54.50 -35.57    0.00   
     off_median -30.00  51.50  -40.00   0.00  -20.00  54.50 -37.00    0.00   
     off_min    -60.00  45.00  -52.00   0.00  -57.00  54.00 -54.00    0.00   
     off_std     14.13   9.19    9.87   0.00   14.01   0.71  10.30    0.00   
     vel_iqr      5.00   0.00   11.00   0.00    3.50   0.00  17.00    0.00   
     vel_max     82.00  55.00   60.00   0.00   85.00  62.00  67.00    0.00   
     vel_mean    77.07  45.00   45.18   0.00   81.25  53.50  47.71    0.00   
     vel_median  77.00  45.00   50.00   0.00   81.00  53.50  48.50    0.00   
     vel_min     70.00  35.00   20.00   0.00   75.00  45.00  17.00    0.00   
     vel_std      3.71  14.14   13.72   0.00    2.89  12.02  12.96    0.00   
43   hits         0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     off_iqr      0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     off_max      0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     off_mean     0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     off_median   0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     off_min      0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     vel_iqr      0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     vel_max      0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     vel_mean     0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     vel_median   0.00   0.00    0.00   0.00    0.00   0.00   0.00    0.00   
     vel_min      0.00

## (4) Try index slicing on final stats df
Excellent info from this document on how to slice and dice this final multiindex structure of stats to pull out what is needed...
* https://github.com/ZaxR/pandas_multiindex_tutorial/blob/master/Pandas%20MultiIndex%20Tutorial.ipynb

In [20]:

# A slicing helper. Works similarly to slicing in Python (e.g. list slicing),
#                   but is inclusive of both the start and stop values.
idx = pd.IndexSlice

print("View all columns where the stat type is 'hits' ")
display(stats_dict.loc[idx[:, 'hits'], ].head(10))

View all columns where the stat type is 'hits' 


,bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,feature,,,,,,,,,,,,,,,,
36,hits,12.0,0.0,16.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,6.0,0.0,1.0,0.0,0.0
38,hits,10.0,5.0,0.0,0.0,16.0,0.0,0.0,16.0,0.0,17.0,0.0,2.0,8.0,0.0,8.0,19.0
42,hits,15.0,2.0,11.0,0.0,16.0,2.0,14.0,0.0,17.0,3.0,13.0,0.0,16.0,1.0,13.0,1.0
43,hits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
49,hits,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
51,hits,16.0,0.0,16.0,3.0,16.0,0.0,16.0,0.0,17.0,0.0,12.0,0.0,16.0,0.0,16.0,0.0


NOTE: the above does away with the need for the 'sf.frequency_table()' code !

In [21]:
print("View info from column/ beat #11 where the stat type is 'hits' ")
display(stats_dict.loc[idx[:, 'hits'], 11].head(10))

View info from column/ beat #11 where the stat type is 'hits' 


note  feature
36    hits       15.0
38    hits        0.0
42    hits       13.0
43    hits        1.0
49    hits        3.0
51    hits       12.0
Name: 11, dtype: float64

In [22]:
print("View IQR (i.e. 'off_iqr') for all instruments, across all beats ...")
display(stats_dict.loc[idx[:, 'off_iqr'], ].head(10))

View IQR (i.e. 'off_iqr') for all instruments, across all beats ...


,bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,feature,,,,,,,,,,,,,,,,
36,off_iqr,18.0,0.0,17.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,10.0,0.0,0.0,0.0,0.0
38,off_iqr,39.0,22.0,0.0,0.0,10.5,0.0,0.0,18.5,0.0,20.0,0.0,0.0,15.5,0.0,31.5,42.5
42,off_iqr,21.0,0.0,12.5,0.0,12.0,0.0,9.0,0.0,15.0,2.5,14.0,0.0,16.0,0.0,18.0,0.0
43,off_iqr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,off_iqr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0
51,off_iqr,16.5,0.0,14.0,3.0,9.5,0.0,14.5,0.0,18.0,0.0,14.0,0.0,18.0,0.0,24.0,0.0


In [23]:
print("View velocity average and std for all instruments, for backbeats 2 & 4 ...")
display(stats_dict.loc[idx[:, ['vel_mean', 'vel_std']], [5, 13]].head(10))

View velocity average and std for all instruments, for backbeats 2 & 4 ...


bar_beat_number      5       13
note feature                   
36   vel_mean      0.00    0.00
     vel_std       0.00    0.00
38   vel_mean    100.06  102.00
     vel_std       6.48   10.04
42   vel_mean     81.25   80.12
     vel_std       2.89    3.77
43   vel_mean      0.00    0.00
49   vel_mean      0.00    0.00
     vel_std       0.00    0.00
51   vel_mean     57.81   55.19